In [ ]:
!pip install watermark

In [2]:
%load_ext watermark

In [3]:
%watermark -a "Rishit Kapoor 21BAI1046" -u -d -v -m

Author: Rishit Kapoor 21BAI1046

Last updated: 2024-04-13

Python implementation: CPython
Python version       : 3.10.12
IPython version      : 7.34.0

Compiler    : GCC 11.4.0
OS          : Linux
Release     : 6.1.58+
Machine     : x86_64
Processor   : x86_64
CPU cores   : 2
Architecture: 64bit



In [ ]:
!pip install kaggle

In [5]:
!mkdir ~/.kaggle

In [6]:
!cp kaggle.json ~/.kaggle/

In [7]:
!chmod 600 ~/.kaggle/kaggle.json

In [8]:
!kaggle datasets download rohanrao/nifty50-stock-market-data

 92% 17.0M/18.4M [00:01<00:00, 16.7MB/s]
100% 18.4M/18.4M [00:01<00:00, 11.5MB/s]


In [ ]:
!unzip '/content/nifty50-stock-market-data.zip' -d '/content/nifty50-stock-market-data'

In [10]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error
import math
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

# Load the data
data = pd.read_csv("/content/nifty50-stock-market-data/NIFTY50_all.csv")

# Select only the 'Close' price column
close_price = data['Close'].values.reshape(-1, 1)

# Normalize the data
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_close_price = scaler.fit_transform(close_price)

# Define the number of time steps
n_steps = 10

# Define a function to prepare data for LSTM
def prepare_data(sequence, n_steps):
    X, y = [], []
    for i in range(len(sequence)):
        # find the end of this pattern
        end_ix = i + n_steps
        # check if we are beyond the sequence
        if end_ix > len(sequence)-1:
            break
        # gather input and output parts of the pattern
        seq_x, seq_y = sequence[i:end_ix], sequence[end_ix]
        X.append(seq_x)
        y.append(seq_y)
    return np.array(X), np.array(y)

# Prepare the data for LSTM
X, y = prepare_data(scaled_close_price, n_steps)

# Reshape data for LSTM [samples, time steps, features]
X = X.reshape((X.shape[0], X.shape[1], 1))

# Define the LSTM model
model = Sequential()
model.add(LSTM(units=50, return_sequences=True, input_shape=(n_steps, 1)))
model.add(LSTM(units=50))
model.add(Dense(units=1))

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error')

# Train the model
model.fit(X, y, epochs=100, batch_size=64)

Epoch 1/100
3675/3675 [==============================] - 26s 6ms/step - loss: 6.5746e-05
Epoch 2/100
3675/3675 [==============================] - 21s 6ms/step - loss: 2.0495e-05
Epoch 3/100
3675/3675 [==============================] - 20s 5ms/step - loss: 1.8402e-05
Epoch 4/100
3675/3675 [==============================] - 20s 6ms/step - loss: 1.6718e-05
Epoch 5/100
3675/3675 [==============================] - 21s 6ms/step - loss: 1.5099e-05
Epoch 6/100
3675/3675 [==============================] - 20s 5ms/step - loss: 1.5524e-05
Epoch 7/100
3675/3675 [==============================] - 21s 6ms/step - loss: 1.5751e-05
Epoch 8/100
3675/3675 [==============================] - 21s 6ms/step - loss: 1.5378e-05
Epoch 9/100
3675/3675 [==============================] - 20s 5ms/step - loss: 1.4997e-05
Epoch 10/100
3675/3675 [==============================] - 21s 6ms/step - loss: 1.4470e-05
Epoch 11/100
3675/3675 [==============================] - 21s 6ms/step - loss: 1.4702e-05
Epoch 12/100
3675/3

In [11]:
# Predict on the last time step of the data
last_sequence = scaled_close_price[-n_steps:]
last_sequence = last_sequence.reshape((1, n_steps, 1))
predicted_close_price = model.predict(last_sequence)

# Print predicted_close_price
print("Predicted Close Price Array:", predicted_close_price)

# Convert the prediction array to a scalar
predicted_close_price_scalar = np.squeeze(predicted_close_price)

# Inverse transform the predicted close price scalar
predicted_close_price_scalar = scaler.inverse_transform([[predicted_close_price_scalar]])

# Compute evaluation metrics
y_true = close_price[-1]
y_pred = predicted_close_price_scalar[-1]
mae = mean_absolute_error(y_true, y_pred)
rmse = math.sqrt(mean_squared_error(y_true, y_pred))
mape = np.mean(np.abs((y_true - y_pred) / y_true)) * 100

print("Sample Prediction Details:")
print("True Close Price:", y_true)
print("Predicted Close Price:", y_pred)
print("Mean Absolute Error (MAE):", mae)
print("Root Mean Squared Error (RMSE):", rmse)
print("Mean Absolute Percentage Error (MAPE):", mape)


1/1 [==============================] - 1s 887ms/step
Predicted Close Price Array: [[0.00509273]]
Sample Prediction Details:
True Close Price: [185.6]
Predicted Close Price: [176.46028106]
Mean Absolute Error (MAE): 9.139718937873852
Root Mean Squared Error (RMSE): 9.139718937873852
Mean Absolute Percentage Error (MAPE): 4.924417531182032


In [12]:
# Stacked Lstm

import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error
import math
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

# Load the data
data = pd.read_csv("/content/nifty50-stock-market-data/NIFTY50_all.csv")

# Select only the 'Close' price column
close_price = data['Close'].values.reshape(-1, 1)

# Normalize the data
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_close_price = scaler.fit_transform(close_price)

# Define the number of time steps
n_steps = 10

# Define a function to prepare data for LSTM
def prepare_data(sequence, n_steps):
    X, y = [], []
    for i in range(len(sequence)):
        # find the end of this pattern
        end_ix = i + n_steps
        # check if we are beyond the sequence
        if end_ix > len(sequence)-1:
            break
        # gather input and output parts of the pattern
        seq_x, seq_y = sequence[i:end_ix], sequence[end_ix]
        X.append(seq_x)
        y.append(seq_y)
    return np.array(X), np.array(y)

# Prepare the data for LSTM
X, y = prepare_data(scaled_close_price, n_steps)

# Reshape data for LSTM [samples, time steps, features]
X = X.reshape((X.shape[0], X.shape[1], 1))

# Define the stacked LSTM model
model = Sequential()
model.add(LSTM(units=50, return_sequences=True, input_shape=(n_steps, 1)))
model.add(LSTM(units=50, return_sequences=True))  # Stacked LSTM layer
model.add(LSTM(units=50))
model.add(Dense(units=1))

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error')

# Train the model
model.fit(X, y, epochs=100, batch_size=128)

Epoch 1/100
1838/1838 [==============================] - 18s 8ms/step - loss: 9.8148e-05
Epoch 2/100
1838/1838 [==============================] - 13s 7ms/step - loss: 3.6402e-05
Epoch 3/100
1838/1838 [==============================] - 13s 7ms/step - loss: 2.0248e-05
Epoch 4/100
1838/1838 [==============================] - 13s 7ms/step - loss: 1.7878e-05
Epoch 5/100
1838/1838 [==============================] - 13s 7ms/step - loss: 1.8709e-05
Epoch 6/100
1838/1838 [==============================] - 13s 7ms/step - loss: 1.6668e-05
Epoch 7/100
1838/1838 [==============================] - 13s 7ms/step - loss: 1.6544e-05
Epoch 8/100
1838/1838 [==============================] - 13s 7ms/step - loss: 1.5872e-05
Epoch 9/100
1838/1838 [==============================] - 13s 7ms/step - loss: 1.6562e-05
Epoch 10/100
1838/1838 [==============================] - 13s 7ms/step - loss: 1.6204e-05
Epoch 11/100
1838/1838 [==============================] - 13s 7ms/step - loss: 1.6504e-05
Epoch 12/100
1838/1

In [13]:
# Predict on the last time step of the data
last_sequence = scaled_close_price[-n_steps:]
last_sequence = last_sequence.reshape((1, n_steps, 1))
predicted_close_price = model.predict(last_sequence)

# Print predicted_close_price
print("Predicted Close Price Array:", predicted_close_price)

# Convert the prediction array to a scalar
predicted_close_price_scalar = np.squeeze(predicted_close_price)

# Inverse transform the predicted close price scalar
predicted_close_price_scalar = scaler.inverse_transform([[predicted_close_price_scalar]])

# Compute evaluation metrics
y_true = close_price[-1]
y_pred = predicted_close_price_scalar[-1]
mae = mean_absolute_error(y_true, y_pred)
rmse = math.sqrt(mean_squared_error(y_true, y_pred))
mape = np.mean(np.abs((y_true - y_pred) / y_true)) * 100

print("Sample Prediction Details:")
print("True Close Price:", y_true)
print("Predicted Close Price:", y_pred)
print("Mean Absolute Error (MAE):", mae)
print("Root Mean Squared Error (RMSE):", rmse)
print("Mean Absolute Percentage Error (MAPE):", mape)


1/1 [==============================] - 1s 845ms/step
Predicted Close Price Array: [[0.00568493]]
Sample Prediction Details:
True Close Price: [185.6]
Predicted Close Price: [195.91589561]
Mean Absolute Error (MAE): 10.31589561104775
Root Mean Squared Error (RMSE): 10.31589561104775
Mean Absolute Percentage Error (MAPE): 5.5581334111248655


In [14]:
#univariate Bidirectional LSTM

import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error
import math
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Bidirectional

# Load the data
data = pd.read_csv("/content/nifty50-stock-market-data/NIFTY50_all.csv")

# Select only the 'Close' price column
close_price = data['Close'].values.reshape(-1, 1)

# Normalize the data
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_close_price = scaler.fit_transform(close_price)

# Define the number of time steps
n_steps = 10

# Define a function to prepare data for LSTM
def prepare_data(sequence, n_steps):
    X, y = [], []
    for i in range(len(sequence)):
        # find the end of this pattern
        end_ix = i + n_steps
        # check if we are beyond the sequence
        if end_ix > len(sequence)-1:
            break
        # gather input and output parts of the pattern
        seq_x, seq_y = sequence[i:end_ix], sequence[end_ix]
        X.append(seq_x)
        y.append(seq_y)
    return np.array(X), np.array(y)

# Prepare the data for LSTM
X, y = prepare_data(scaled_close_price, n_steps)

# Reshape data for LSTM [samples, time steps, features]
X = X.reshape((X.shape[0], X.shape[1], 1))

# Define the Bidirectional LSTM model
model = Sequential()
model.add(Bidirectional(LSTM(units=50, input_shape=(n_steps, 1))))
model.add(Dense(units=1))

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error')

# Train the model
model.fit(X, y, epochs=100, batch_size=128)

Epoch 1/100
1838/1838 [==============================] - 13s 5ms/step - loss: 9.2102e-05
Epoch 2/100
1838/1838 [==============================] - 10s 5ms/step - loss: 2.9572e-05
Epoch 3/100
1838/1838 [==============================] - 10s 5ms/step - loss: 1.9448e-05
Epoch 4/100
1838/1838 [==============================] - 9s 5ms/step - loss: 1.6003e-05
Epoch 5/100
1838/1838 [==============================] - 10s 5ms/step - loss: 1.4539e-05
Epoch 6/100
1838/1838 [==============================] - 10s 5ms/step - loss: 1.4058e-05
Epoch 7/100
1838/1838 [==============================] - 10s 5ms/step - loss: 1.3755e-05
Epoch 8/100
1838/1838 [==============================] - 10s 5ms/step - loss: 1.3438e-05
Epoch 9/100
1838/1838 [==============================] - 10s 5ms/step - loss: 1.3140e-05
Epoch 10/100
1838/1838 [==============================] - 10s 5ms/step - loss: 1.3220e-05
Epoch 11/100
1838/1838 [==============================] - 10s 5ms/step - loss: 1.3139e-05
Epoch 12/100
1838/18

In [15]:
# Predict on the last time step of the data
last_sequence = scaled_close_price[-n_steps:]
last_sequence = last_sequence.reshape((1, n_steps, 1))
predicted_close_price = model.predict(last_sequence)

# Print predicted_close_price
print("Predicted Close Price Array:", predicted_close_price)

# Convert the prediction array to a scalar
predicted_close_price_scalar = np.squeeze(predicted_close_price)

# Inverse transform the predicted close price scalar
predicted_close_price_scalar = scaler.inverse_transform([[predicted_close_price_scalar]])

# Compute evaluation metrics
y_true = close_price[-1]
y_pred = predicted_close_price_scalar[-1]
mae = mean_absolute_error(y_true, y_pred)
rmse = math.sqrt(mean_squared_error(y_true, y_pred))
mape = np.mean(np.abs((y_true - y_pred) / y_true)) * 100

print("Sample Prediction Details:")
print("True Close Price:", y_true)
print("Predicted Close Price:", y_pred)
print("Mean Absolute Error (MAE):", mae)
print("Root Mean Squared Error (RMSE):", rmse)
print("Mean Absolute Percentage Error (MAPE):", mape)


1/1 [==============================] - 1s 570ms/step
Predicted Close Price Array: [[0.00510431]]
Sample Prediction Details:
True Close Price: [185.6]
Predicted Close Price: [176.8408869]
Mean Absolute Error (MAE): 8.759113100916153
Root Mean Squared Error (RMSE): 8.759113100916153
Mean Absolute Percentage Error (MAPE): 4.719349731097065
